# Maven Airlines Case Study for Passenger Satisfaction

## Introduction : 

For this challenge, you'll be assuming the role of Senior Data Analyst for Maven Airlines, a US-based airline headquartered in Boston, Massachusetts. The latest passenger survey results just came in and it looks like the satisfaction rate dipped under 50% for the first time ever. The leadership team needs to take action fast, so they've brought you in to analyze the data and find the key areas to focus on for getting back on track.

Your task is to recommend a data-driven strategy for increasing Maven Airlines' satisfaction rate, and present it in the form of a single page report or dashboard.

## 1 - Specifications

There I will be using the Mavenanalytics data set, as it is made public I assume that everybody is authorized to work on it and I also assume that there is no issue with bias or credibility of this datas.

## 2 - Preliminary questions 

I wanted to raise as many questions as possible covering all the different criterias of the data set. Basically I will proceed step by step :<br>

   - <b>What is the ratio of satisfied passengers to dissatisfied passengers?</b> This one is basic although it remains essential for further analysis <br><br>
    
   - <b>What is the satisfaction rate by gender stats?</b> By differentiating the gender I might encounter useful results and eventually consider it as a source of improvement for the future recommendations.<br><br>
    
   - <b>What is the satisfaction rate by customer type of travel and class ?</b> There I will be splitting my results and I could later on focus on what type of class efforts should be made and what kind of travel are most likely required to be worked on<br><br>
   - <b>What is the satisfaction rate by flight distance ?</b> There I am going to categorize the travel under 3 categories and use it to increase the granularity of my analysis<br><br>
   - Finally, I will also study each features by type of passengers in order to retrieve the factors that affects positively or negatively the customer satisfaction.

## 3 - Processing the data

The data set file provided by MavenAnalytics is an excel CSV file. I had several option to exploit this data set, but in this case I am going to use Python and the famous libraries Pandas,Numpy and Seaborn to visualize all my data. Here my objective is not to provide a one page sumary of all the results but rather a paper which will tackle all the questions I have raised and bring proper explanations and solution as a conclusion. 

First I will import all the required libraries and I will also convert the CSV file as a dataset. 